# KMT2D / Kabuki Syndrome 1 due to KMT2D deficiency 
##### OMIM: 147920, MONDO:0007843



##### Clinical Synopsis (USIDNET2024)

Typical facial abnormalities, cleft or high arched palate, skeletal abnormalities, 
short stature, intellectual disability, congenital heart defects, recurrent 
infections (otitis media, pneumonia) in 50% of patients. Autoimmunity may be present
- Group: Combined immunodeficiencies with associated or syndromic features
- Subgroup: Other Combined immunodeficiencies wwith syndromic features 









In [6]:
cohort_name = 'KMT2D'
tx_id = 'NM_003482.4'
px_id = 'NP_003473.3' 

In [7]:
import hpotk
store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()

In [8]:
from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store('0.1.18') as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(cohort_name))

In [9]:
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets
cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(  
    phenopackets=phenopackets,
    cohort_creator=cohort_creator,
)

Individuals Processed: 100%|██████████| 65/65 [01:42<00:00,  1.57s/individuals]


In [10]:
validation.summarize()  

Validated under none policy
Showing errors and warnings
Phenopackets
  patient #46
    phenotype-features
     warnings:
     ·#3 HP:0000368 is an obsolete identifier for Posteriorly rotated ears. Replace HP:0000368 with the primary term ID HP:0000358


In [11]:
from IPython.display import HTML, display

from gpsea.view import CohortViewable
viewer = CohortViewable(hpo)
report = viewer.process(cohort=cohort, transcript_id=tx_id)
display(HTML(report))

HPO Term,ID,Seen in n individuals
Depressed nasal tip,HP:0000437,36
Prominent fingertip pads,HP:0001212,30
Global developmental delay,HP:0001263,30
Long palpebral fissure,HP:0000637,25
Macrotia,HP:0000400,24
Highly arched eyebrow,HP:0002553,22
Intellectual disability,HP:0001249,21
Eversion of lateral third of lower eyelids,HP:0007655,18
Cleft palate,HP:0000175,17
Hearing impairment,HP:0000365,16
